In [1]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
from autogluon.tabular import TabularPredictor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
from typing import Dict, List, Optional, Tuple
from sklearn.model_selection import ParameterGrid

In [2]:

n_estimators = [int(x) for x in np.linspace(start = 10, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 3)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [ 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [ 4]
# Method of selecting samples for training each tree
bootstrap = [True]# Create the random grid
    

In [3]:
random_grid =  { 
                'randomforest__n_estimators': n_estimators,
               #'max_features': max_features,
               'randomforest__max_depth': max_depth,
               'randomforest__min_samples_split': min_samples_split,
               'randomforest__min_samples_leaf': min_samples_leaf,
               'randomforest__bootstrap': bootstrap
        }

In [17]:
pg = ParameterGrid(random_grid)
len(pg)

40

In [5]:
random_grid

{'randomforest__n_estimators': [10,
  152,
  294,
  436,
  578,
  720,
  862,
  1005,
  1147,
  1289,
  1431,
  1573,
  1715,
  1857,
  2000],
 'randomforest__max_depth': [10,
  20,
  30,
  40,
  50,
  60,
  70,
  80,
  90,
  100,
  110,
  None],
 'randomforest__min_samples_split': [2, 5, 10],
 'randomforest__min_samples_leaf': [1, 2, 4],
 'randomforest__bootstrap': [True, False]}

In [9]:
original_data_train = pd.read_csv("train_data.csv",index_col=0)
original_labels_train = pd.read_csv("train_labels.csv",index_col=0)
original_data_test = pd.read_csv("test_data.csv",index_col=0)
original_labels_test = pd.read_csv("test_labels.csv",index_col=0)


In [10]:
original_labels_train = original_labels_train.sort_index()
original_labels_train.reset_index(drop=True, inplace=True)

original_data_train = original_data_train.copy().sort_index()
original_data_train.reset_index(drop=True, inplace=True)

In [11]:
original_data_test = original_data_test.sort_index()
original_data_test.reset_index(drop=True, inplace=True)

original_labels_test = original_labels_test.sort_index()
original_labels_test.reset_index(drop=True, inplace=True)

In [12]:
original_data_train.sort_index()

,V1,V2,V3,V4,V5,V6,V7
0,1.0,19.0,0.0,19.0,8.0,8.00,46.10
1,2.0,0.0,20.0,19.0,8.0,22.47,46.10
2,4.0,19.0,8.0,8.0,42.0,35.74,46.10
3,0.0,0.0,0.0,0.0,12.0,12.00,37.66
4,1.0,7.0,0.0,7.0,7.0,7.00,37.66
...,...,...,...,...,...,...,...
8169,2.0,0.0,44.0,18.0,39.0,18.68,49.19
8170,3.0,18.0,39.0,22.0,33.0,7.21,49.19
8171,4.0,22.0,33.0,22.0,20.0,13.00,49.19
8172,5.0,22.0,20.0,18.0,14.0,7.21,49.19


In [13]:
corrupted_data_train = pd.read_csv("train_data_corrupted.csv",index_col=0)
corrupted_data_test = pd.read_csv("test_data_corrupted.csv",index_col=0)


In [14]:
corrupted_data_test

,V1,V2,V3,V4,V5,V6,V7
2045,0.0,0.0,0.0,0.0,29.0,29.00,31.38
4816,0.0,0.0,0.0,0.0,25.0,25.00,NaN
5012,5.0,14.0,14.0,0.0,6.0,16.12,NaN
33,2.0,0.0,6.0,10.0,7.0,10.05,26.00
2926,2.0,0.0,15.0,28.0,14.0,28.02,48.01
...,...,...,...,...,...,...,...
8131,2.0,0.0,14.0,9.0,16.0,9.22,NaN
738,4.0,15.0,10.0,3.0,23.0,17.69,27.46
538,1.0,12.0,0.0,13.0,11.0,11.05,NaN
7200,1.0,0.0,13.0,2.0,20.0,7.28,26.63


In [15]:
original_labels_train.index.equals(corrupted_data_train.index)

False

In [16]:
original_data_train

,V1,V2,V3,V4,V5,V6,V7
0,1.0,19.0,0.0,19.0,8.0,8.00,46.10
1,2.0,0.0,20.0,19.0,8.0,22.47,46.10
2,4.0,19.0,8.0,8.0,42.0,35.74,46.10
3,0.0,0.0,0.0,0.0,12.0,12.00,37.66
4,1.0,7.0,0.0,7.0,7.0,7.00,37.66
...,...,...,...,...,...,...,...
8169,2.0,0.0,44.0,18.0,39.0,18.68,49.19
8170,3.0,18.0,39.0,22.0,33.0,7.21,49.19
8171,4.0,22.0,33.0,22.0,20.0,13.00,49.19
8172,5.0,22.0,20.0,18.0,14.0,7.21,49.19


In [17]:
original_labels_test.index.equals(original_data_test.index)

True

In [18]:
def preprocess_and_transform_train(train_data, train_labels, categorical_columns, numerical_columns):
        # Define the categorical preprocessing pipeline
        categorical_preprocessing = Pipeline(
            [
                ('mark_missing', SimpleImputer(strategy='most_frequent')),
                ('one_hot_encode', OneHotEncoder(handle_unknown='ignore',sparse= False))
            ]
        )

        # Define the numerical preprocessing pipeline
        numerical_preprocessing = Pipeline(
            [
                ('mark_missing', SimpleImputer(strategy='mean')),
                ('scaling', StandardScaler())
            ]
        )

        # Define the column transformer
        feature_transformation = ColumnTransformer(transformers=[
                ('categorical_features', categorical_preprocessing, categorical_columns),
                ('scaled_numeric', numerical_preprocessing, numerical_columns)
            ]
        )
        
        # Sort and reset indices for train_labels and train_data
        labels = train_labels.sort_index()
        labels.reset_index(drop=True, inplace=True)

        train_data_sorted = train_data.copy().sort_index()
        train_data_sorted.reset_index(drop=True, inplace=True)

        # Transform the data
        
        if not categorical_columns and not numerical_columns:
            raise ValueError("categorical_columns list and numerical_columns list are empty. Feature Prep. Pipeline needs imput!")
        else:
            transformed_data = pd.DataFrame(feature_transformation.fit_transform(train_data_sorted))
        
        
        
        # Add the labels to the transformed data

        transformed_data["train_label"] = labels
        #transformed_data.to_csv("debug.csv")
        return transformed_data, feature_transformation, labels
    


In [19]:
transformed_data, feature_transformation, labels = preprocess_and_transform_train(corrupted_data_train.copy(),original_labels_train.copy(),[],["V1","V2","V3","V4","V5","V6","V7"])

In [20]:
transformed_data

,0,1,2,3,4,5,6,train_label
0,-0.709000,1.469875,-1.078695,0.993369,-0.991846,-0.941238,7.234685e-16,1
1,-0.126024,-0.674952,0.337925,0.993369,-0.991846,0.904366,7.234685e-16,1
2,1.039928,1.469875,-0.512047,-0.134894,1.594910,2.596913,7.234685e-16,1
3,-1.291975,-0.674952,-1.078695,-0.955448,-0.687522,-0.431050,7.234685e-16,1
4,-0.709000,0.115247,-1.078695,-0.237463,-1.067927,-1.068785,7.234685e-16,1
...,...,...,...,...,...,...,...,...
8169,-0.126024,-0.674952,2.037868,0.890799,1.366666,0.420963,7.234685e-16,3
8170,0.456952,1.356990,1.683713,1.301077,0.910180,-1.042000,7.234685e-16,3
8171,1.039928,1.808532,1.258727,1.301077,-0.078873,-0.303503,7.234685e-16,3
8172,1.622904,1.808532,0.337925,0.890799,-0.535360,-1.042000,7.234685e-16,3


In [21]:
excluded_model_types = ['KNN','CAT','FASTAI','XT','GBM',"RF","NN_TORCH"]

model = TabularPredictor(label="train_label").fit(transformed_data,time_limit=60,excluded_model_types=excluded_model_types,hyperparameters={ 'XGB':{}},feature_generator=None)
print("---------------------------")
print(model.leaderboard())
model.delete_models(models_to_keep='best', dry_run=False)

No path specified. Models will be saved in: "AutogluonModels\ag-20240906_124807"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "AutogluonModels\ag-20240906_124807"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.8.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          12
Me

---------------------------
                 model  score_val eval_metric  pred_time_val  fit_time  \
0              XGBoost   0.845966    accuracy       0.122001  9.756125   
1  WeightedEnsemble_L2   0.845966    accuracy       0.123000  9.761124   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                0.122001           9.756125            1       True   
1                0.001000           0.004999            2       True   

   fit_order  
0          1  
1          2  


In [25]:
model.model_best.

'WeightedEnsemble_L2'

In [14]:
def preprocess_and_transform_test(tets_data, test_labels, feature_transformer,categorical_columns,numerical_columns):
            # Define the categorical preprocessing pipeline
            
            
            # Sort and reset indices for test_labels and tets_data
            labels = test_labels.sort_index()
            labels.reset_index(drop=True, inplace=True)

            tets_data_sorted = tets_data.copy().sort_index()
            tets_data_sorted.reset_index(drop=True, inplace=True)

            # Transform the data

            #categorical_columns = []
            #numerical_columns =["V1","V2","V3","V4","V5","V6"]
            if not categorical_columns and not numerical_columns:
                 raise ValueError("categorical_columns list and numerical_columns list are empty. Feature Prep. Pipeline needs imput!")
            else:
                transformed_data = pd.DataFrame(feature_transformer.transform(tets_data_sorted))

            
            
            # Add the labels to the transformed data


            return transformed_data, labels



In [15]:
corrupted_test, test_labels_sorted = preprocess_and_transform_test(corrupted_data_test.copy(),original_labels_test.copy(), feature_transformation,[],["V1","V2","V3","V4","V5","V6","V7"])

In [16]:
corrupted_test

,0,1,2,3,4,5,6
0,-1.291975,-0.674952,-1.078695,-0.955448,-0.078873,0.589325,7.234685e-16
1,0.456952,-0.674952,0.337925,-0.134894,1.594910,1.024260,7.234685e-16
2,0.456952,-0.674952,-0.228723,-0.852879,1.062342,0.974517,7.234685e-16
3,1.622904,-0.562067,1.400389,-0.750309,1.214504,-1.675908,7.234685e-16
4,-1.291975,-0.674952,-1.078695,-0.955448,-0.078873,0.589325,7.234685e-16
...,...,...,...,...,...,...,...
2039,1.039928,0.453904,0.621248,-0.955448,-0.154954,-0.535639,7.234685e-16
2040,-0.709000,-0.562067,0.762910,-0.852879,0.986261,-0.941238,7.234685e-16
2041,1.622904,0.792561,0.621248,0.172814,-0.078873,-1.391478,7.234685e-16
2042,-0.709000,-0.674952,0.479587,-0.955448,1.747072,0.844419,1.303944e+00


In [17]:
corrupted_predictions  = model.predict(corrupted_test)

In [18]:
f1_score(original_labels_test, corrupted_predictions, average="macro")

0.7750415246097008

## Imputation experiment

In [4]:
corrupted_data_train = pd.read_csv("train_data_corrupted.csv",index_col=0)
corrupted_labels= pd.read_csv("train_labels.csv",index_col=0)

In [5]:
rf = RandomForestRegressor()
categorical_preprocessing = Pipeline(
            [
                
                ('encode', OneHotEncoder(handle_unknown='ignore'))
            ]
        )

numeric_preprocessing = Pipeline(
            [
                ('scale',  StandardScaler()),
            ]
        )
feature_transformation = ColumnTransformer(transformers=[
                    ('categorical_features', categorical_preprocessing,[]),
                    ('numerical_features', numeric_preprocessing, ["V1","V2","V3","V4","V5","V6"])
                ]
            )

pipeline = Pipeline(
                [
                    ('features', feature_transformation),
                    #('scale',  StandardScaler()),
                    ('randomforest',rf )
                ]
            )


In [75]:
pipeline

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('categorical_features',
                                                  Pipeline(steps=[('encode',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  []),
                                                 ('numerical_features',
                                                  Pipeline(steps=[('scale',
                                                                   StandardScaler())]),
                                                  ['V1', 'V2', 'V3', 'V4', 'V5',
                                                   'V6'])])),
                ('randomforest', RandomForestRegressor())])

In [71]:
corrupted_data_train = corrupted_data_train.sort_index()
corrupted_data_train.reset_index(drop=True, inplace=True)

corrupted_labels = corrupted_labels.sort_index()
corrupted_labels.reset_index(drop=True, inplace=True)




In [6]:
missing_mask = corrupted_data_train["V7"].isna()

df_train = corrupted_data_train[~missing_mask]
df_train_labels= corrupted_labels[~missing_mask]

df_with_missing_labels= corrupted_labels[missing_mask]
df_with_missing = corrupted_data_train[missing_mask]

In [107]:
corrupted_data_train.loc[~missing_mask,"V7"].drop(columns="V7")

0       30.795852
1       37.129324
2       61.992778
3       28.362881
4       23.695214
          ...    
8169    57.186841
8170    55.682896
8171    52.205473
8172    35.023878
8173    44.248362
Name: V7, Length: 8174, dtype: float64

In [77]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = pipeline, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(df_train.iloc[:,:-1], df_train.iloc[:,-1:])

Fitting 3 folds for each of 20 candidates, totalling 60 fits


C:\Users\Petro\AppData\Roaming\Python\Python38\site-packages\sklearn\pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('features',
                                              ColumnTransformer(transformers=[('categorical_features',
                                                                               Pipeline(steps=[('encode',
                                                                                                OneHotEncoder(handle_unknown='ignore'))]),
                                                                               []),
                                                                              ('numerical_features',
                                                                               Pipeline(steps=[('scale',
                                                                                                StandardScaler())]),
                                                                               ['V1',
                                                                                'V2',
                                                                                'V3',
                                                                                'V4',
                                                                                'V5',
                                                                                'V6'])])),
                                             ('randomforest',
                                              RandomForestRegressor())]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'randomforest__bootstrap': [True,
                                                                    False],
                                        'randomforest__max_depth': [10, 20, 30,
                                                                    40, 50, 60,
                                                                    70, 80, 90,
                                                                    100, 110,
                                                                    None],
                                        'randomforest__min_samples_leaf': [1, 2,
                                                                           4],
                                        'randomforest__min_samples_split': [2,
                                                                            5,
                                                                            10],
                                        'randomforest__n_estimators': [10, 152,
                                                                       294, 436,
                                                                       578, 720,
                                                                       862,
                                                                       1005,
                                                                       1147,
                                                                       1289,
                                                                       1431,
                                                                       1573,
                                                                       1715,
                                                                       1857,
                                                                       2000]},
                   random_state=42, verbose=2)

In [81]:
df_train_imp, feature_transformation, labels = preprocess_and_transform_train(df_train.iloc[:,:-1],df_train.iloc[:,-1:],[],["V1","V2","V3","V4","V5","V6"])


In [8]:
excluded_model_types = ['XGB','CAT','FASTAI','XT','GBM',"KNN","NN_TORCH"]


imputer = TabularPredictor(label="train_label").fit(df_train_imp,time_limit=360,excluded_model_types=excluded_model_types,hyperparameters={ 'RF':{}},feature_generator=None)
print("---------------------------")
print(imputer.leaderboard())
imputer.delete_models(models_to_keep='best', dry_run=False)

No path specified. Models will be saved in: "AutogluonModels\ag-20240906_124743"


NameError: name 'df_train_imp' is not defined

In [83]:
randomForest= rf_random.best_estimator_

In [84]:
missing_mask = corrupted_data_train["V7"].isna()
amount_missing_in_columns = missing_mask.sum()
column= "V7"



In [85]:
imputer_predict_corrupted , labels_imput = preprocess_and_transform_test(df_with_missing,corrupted_labels,feature_transformation,[],["V1","V2","V3","V4","V5","V6"])

In [86]:
imputer_predict_corrupted

,0,1,2,3,4,5
0,-0.693319,1.496686,-0.982022,1.114191,-0.819993,-0.741631
1,-0.112426,-0.667600,0.461744,1.114191,-0.819993,1.122244
2,1.049359,1.496686,-0.404516,-0.055442,1.817796,2.831549
3,-1.274212,-0.667600,-0.982022,-0.906083,-0.509665,-0.226393
4,-0.693319,0.129769,-0.982022,-0.161772,-0.897575,-0.870441
...,...,...,...,...,...,...
4082,-0.112426,-0.667600,2.194263,1.007860,1.585050,0.634056
4083,0.468467,1.382776,1.833321,1.433181,1.119558,-0.843391
4084,1.049359,1.838415,1.400192,1.433181,0.110991,-0.097583
4085,1.630252,1.838415,0.461744,1.007860,-0.354501,-0.843391


In [87]:
to_be_imputed=randomForest.predict(df_with_missing.iloc[:,:-1])

In [88]:
len(to_be_imputed)

4087

In [89]:
missing_mask = corrupted_data_train["V7"].isna()
amount_missing_in_columns = missing_mask.sum()
column= "V7"



In [90]:
corrupted_data_train.loc[missing_mask, column] = to_be_imputed

In [91]:
missing_mask = corrupted_data_train["V7"].isna()
missing_mask

0       False
1       False
2       False
3       False
4       False
        ...  
8169    False
8170    False
8171    False
8172    False
8173    False
Name: V7, Length: 8174, dtype: bool

In [92]:
transformed_data, feature_transformation, labels = preprocess_and_transform_train(corrupted_data_train.copy(),original_labels_train.copy(),[],["V1","V2","V3","V4","V5","V6","V7"])


In [40]:
transformed_data

,0,1,2,3,4,5,6,train_label
0,-0.709000,1.469875,-1.078695,0.993369,-0.991846,-0.941238,-0.721151,1
1,-0.126024,-0.674952,0.337925,0.993369,-0.991846,0.904366,-0.223607,1
2,1.039928,1.469875,-0.512047,-0.134894,1.594910,2.596913,1.636706,1
3,-1.291975,-0.674952,-1.078695,-0.955448,-0.687522,-0.431050,-0.909962,1
4,-0.709000,0.115247,-1.078695,-0.237463,-1.067927,-1.068785,-1.285475,1
...,...,...,...,...,...,...,...,...
8169,-0.126024,-0.674952,2.037868,0.890799,1.366666,0.420963,1.298766,3
8170,0.456952,1.356990,1.683713,1.301077,0.910180,-1.042000,1.208980,3
8171,1.039928,1.808532,1.258727,1.301077,-0.078873,-0.303503,0.917265,3
8172,1.622904,1.808532,0.337925,0.890799,-0.535360,-1.042000,-0.414875,3


In [93]:
excluded_model_types = ['KNN','CAT','FASTAI','XT','GBM',"RF","NN_TORCH"]


imputed_model = TabularPredictor(label="train_label").fit(transformed_data,time_limit=60,excluded_model_types=excluded_model_types,hyperparameters={ 'XGB':{}},feature_generator=None)
print("---------------------------")
print(imputed_model.leaderboard())
imputed_model.delete_models(models_to_keep='best', dry_run=False)

No path specified. Models will be saved in: "AutogluonModels\ag-20240904_105020"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "AutogluonModels\ag-20240904_105020"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.8.19
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          12
Me

---------------------------
                 model  score_val eval_metric  pred_time_val  fit_time  \
0              XGBoost   0.833741    accuracy       0.137999  8.826999   
1  WeightedEnsemble_L2   0.833741    accuracy       0.138998  8.830996   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                0.137999           8.826999            1       True   
1                0.000999           0.003997            2       True   

   fit_order  
0          1  
1          2  


In [94]:
test_data_corrupted= pd.read_csv("test_data_corrupted.csv",index_col=0)
original_labels_test = pd.read_csv("test_labels.csv",index_col=0)

In [95]:
missing_mask_test_corrupted = test_data_corrupted["V7"].isna()
amount_missing_in_columns = missing_mask.sum()
column= "V7"
df_with_missing_corrupted_test = test_data_corrupted[missing_mask_test_corrupted]
to_be_imputed_corrupted_test=randomForest.predict(df_with_missing_corrupted_test.iloc[:,:-1])
test_data_corrupted.loc[missing_mask_test_corrupted, column] = to_be_imputed_corrupted_test

In [96]:
imputed_test, imputed_labels = preprocess_and_transform_test(test_data_corrupted.copy(),original_labels_test.copy(),feature_transformation,[],["V1","V2","V3","V4","V5","V6"])

In [97]:
imputed_predictions  = imputed_model.predict(imputed_test)

In [98]:
f1_score(imputed_labels, imputed_predictions, average="macro")

0.7909777972675534

In [331]:
imputed = pd.read_csv("train_imputed.csv",index_col=0)
imputed

,V1,V2,V3,V4,V5,V6,V7
2107,3.0,1.0,27.0,23.0,27.0,22.00,35.470000
1400,4.0,7.0,25.0,15.0,30.0,9.43,33.540000
9117,0.0,0.0,0.0,0.0,20.0,20.00,29.410000
5369,5.0,0.0,6.0,18.0,0.0,18.97,41.693000
9215,0.0,0.0,0.0,0.0,22.0,22.00,31.620000
...,...,...,...,...,...,...,...
5734,0.0,28.0,0.0,0.0,10.0,29.73,58.366600
5191,4.0,19.0,21.0,0.0,6.0,24.21,49.262613
5390,0.0,0.0,0.0,0.0,36.0,36.00,61.850000
860,1.0,22.0,0.0,22.0,7.0,7.00,33.488200


In [332]:
imputed = pd.read_csv("test_imputed.csv",index_col=0)
imputed

,V1,V2,V3,V4,V5,V6,V7
2045,0.0,0.0,0.0,0.0,29.0,29.00,31.380000
4816,0.0,0.0,0.0,0.0,25.0,25.00,54.712268
5012,5.0,14.0,14.0,0.0,6.0,16.12,44.948460
33,2.0,0.0,6.0,10.0,7.0,10.05,26.000000
2926,2.0,0.0,15.0,28.0,14.0,28.02,48.010000
...,...,...,...,...,...,...,...
8131,2.0,0.0,14.0,9.0,16.0,9.22,32.055000
738,4.0,15.0,10.0,3.0,23.0,17.69,27.460000
538,1.0,12.0,0.0,13.0,11.0,11.05,43.311334
7200,1.0,0.0,13.0,2.0,20.0,7.28,26.630000
